# Etude de l'ARCEP

Nos questions : 

## Rapprochement
- combien de lignes immeubles dans le fichier de l'ARCEP ?
- combien de lignes tombent sur un bâtiment RNB ?

## Enrichissement lien bati-adresses RNB
- en prenant un échantillon de match, peut-on améliorer le lien bati ↔ adresse en se basant sur les adresses fournies par l’arcep ?
- combien de points ARCEP tombent sur des bâtiment RNB sans adresse

## Création de bâtiments RNB
- en prenant un échantillon de X (50?) point ARCEP tombant loin d’un bâtiment RNB : 
  - combien devraient donner lieu à la création automatique d’un bâtiment ?
  - combien sont en doute ?
    - est-ce qu’il y a un champs qui les distingue ?
  - combien sont des erreurs ?

In [1]:
import os
import csv
from pprint import pprint

from batid.services.guess_bdg_new import Guesser, ClosestFromPointHandler

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
data_path = "arcep_immeuble_t4_2024.csv"
guesses_path = "guesses.json"

**Question : combien de lignes immeubles dans le fichier de l'ARCEP ?**

In [2]:
with open(data_path, "r") as f:
    row_count = sum(1 for row in f)

    print(row_count)

24762753


**Question : combien de lignes tombent sur un bâtiment RNB ?**

In [20]:
def row_to_input(row):
    
    return {
        "ext_id": f"{row['imb_id']}",
        "lat": float(row["y"]),
        "lng": float(row["x"])
    }
    

On prend un échantillon de 500 000 lignes

In [21]:
with open(data_path, "r") as f:
    reader = csv.DictReader(f)

    inputs = []
    
    ext_ids = set()
    
    c = 0
    for row in reader:
        c += 1
        
        if c == 1:
            pprint(row)
        
        if row['imb_id'] not in ext_ids:
        
            inputs.append(row_to_input(row))
            ext_ids.add(row['imb_id'])

        if c > 500000:
            break

{'batiment': '',
 'catg_loc_imb': 'individuel',
 'code_insee': '57669',
 'code_l331': 'FAFO',
 'code_poste': '57450',
 'cp_no_voie': '',
 'date_completude': '',
 'date_completude_manquante': 't',
 'geom_mod': 'f',
 'imb_etat': 'deploye',
 'imb_id': 'THED1000260',
 'nom_com': 'Théding',
 'nom_voie': 'D EBRING',
 'num_voie': '35',
 'pm_etat': 'deploye',
 'pm_ref': 'N057CF2_S034',
 'type_imb': 'PA',
 'type_voie': 'RUE',
 'x': '6.900318335855359',
 'y': '49.12606398675791'}


In [22]:
g = Guesser(batch_size=2000)
g.handlers = [ClosestFromPointHandler()]
g.create_work_file(inputs,guesses_path)

In [23]:
g.guess_work_file(guesses_path)

  0%|          | 0/250 [00:00<?, ?it/s]

In [24]:
g.report()

-- Report --
Number of rows: 499971
Number of match: 457172 (91.44%)

-- finished_steps --
Rows with finished_steps closest_from_point: 499971 (100.00%)
Rows with empty finished_steps: 0 (0.00%)

-- match_reasons : absolute --
match_reason
point_on_bdg            440076
isolated_closest_bdg     17096
Name: count, dtype: int64

-- match_reasons : % --
match_reason
point_on_bdg            88.020305
isolated_closest_bdg     3.419398
Name: count, dtype: float64

-- Inputs --


Sur les 500000 lignes de l'échantillon, on arrive à associer 457172 identifiants RNB (91,44%).
En extrapolant aux 24762753 lignes du fichier **on peut probablement arriver à fournir 22 643 061 identifiants.**


**Question : en prenant un échantillon de match, peut-on améliorer le lien bati ↔ adresse en se basant sur les adresses fournies par l’arcep ?**

In [21]:
# We forgot to add addresses to the inputs. :(
# We add them

g = Guesser()
g.load_work_file(guesses_path)

with open(data_path, "r") as f:
    reader = csv.DictReader(f)
    
    c = 0
    for row in reader:
        
        c += 1
        if c % 200_000 == 0:
            print(c)
        
        imb_id = row['imb_id']
        address = f"{row['num_voie']} {row['type_voie']} {row['nom_voie']} {row['code_poste']} {row['nom_com']}"
        
        
        if g.guesses.get(imb_id):
            g.guesses[imb_id]["input"]["address"] = address
        
    
    g.save_work_file(guesses_path)



200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
2000000
2200000
2400000
2600000
2800000
3000000
3200000
3400000
3600000
3800000
4000000
4200000
4400000
4600000
4800000
5000000
5200000
5400000
5600000
5800000
6000000
6200000
6400000
6600000
6800000
7000000
7200000
7400000
7600000
7800000
8000000
8200000
8400000
8600000
8800000
9000000
9200000
9400000
9600000
9800000
10000000
10200000
10400000
10600000
10800000
11000000
11200000
11400000
11600000
11800000
12000000
12200000
12400000
12600000
12800000
13000000
13200000
13400000
13600000
13800000
14000000
14200000
14400000
14600000
14800000
15000000
15200000
15400000
15600000
15800000
16000000
16200000
16400000
16600000
16800000
17000000
17200000
17400000
17600000
17800000
18000000
18200000
18400000
18600000
18800000
19000000
19200000
19400000
19600000
19800000
20000000
20200000
20400000
20600000
20800000
21000000
21200000
21400000
21600000
21800000
22000000
22200000
22400000
22600000
22800000
23000000
23200000
23400000


In [31]:
# Get a random set of matches with their address
g = Guesser()
g.load_work_file(guesses_path)

import random

set_size = 30
random_matches = []
for ext_id, guess in g.guesses.items():
    
    if len(guess['matches']) > 0:
    
        # one chance on 50
        if random.randint(1, 10000) == 42:
            random_matches.append({
                'imb_id': ext_id,
                'rnb_id': guess["matches"][0],
                'address': guess["input"]["address"]
            })

            if len(random_matches) >= set_size:
                break
            

In [32]:
import pandas as pd
from IPython.display import HTML, display

df = pd.DataFrame(random_matches)

display(HTML(df.to_html()))



,imb_id,rnb_id,address
0,IMB/66037/X/05M1,7P2JBB2YF7XD,0 Avenue des Anneaux du Roussillon 66140 Canet-en-Roussillon
1,IMB/01264/X/0096,KYKNVBB57SA3,544 Route de Trévoux 01310 Montracol
2,IMB/01301/X/00BT,TA8FS8HMZQPT,176 Rue de la Croix Rouge 01310 Polliat
3,IMB/02371/X/009B,63DEQZ9C65CR,63 Rue de Bellevue 02100 Harly
4,IMB/02691/S/09PO,NQMAMDPH4DMH,35 Boulevard du Docteur Camille Guérin 02100 Saint-Quentin
5,IMB/02691/S/0CDV,4YE6B5SFERD6,94 Rue du Sentier 02100 Saint-Quentin
6,IMB/02691/X/0FOS,TJK8E7FCJG89,34 Rue Verlaine 02100 Saint-Quentin
7,IMB/03023/X/01HD,NPVWV94GYXJ6,26 Chemin de la Varenne du Léry 03700 Bellerive-sur-Allier
8,IMB/02722/X/04RU,NWSJJV5ZC19V,27 Avenue de Laon 02200 Soissons
9,IMB/03009/X/002M,7K3ZT3YGHD6T,7 Route de Bagneux 03460 Aubigny


J'ai passé en revue les adresses proposées par l'ARCEP [sur Notion](https://www.notion.so/referentielnationaldesbatiments/Etudier-int-r-t-ARCEP-14d43ec9d11e8079baf7fed793e84493?pvs=4).

- sur 30 cas, l'adresse RNB et l'adresse ARCEP diffèrent 5 fois
- pour ces 5 cas : 
  - le RNB a raison 3 fois
  - pour les 2 cas restants:
    - le RNB n'avait pas d'adresse associée à ces bâtiments
    - l'Arcep avait tord une fois et raison une fois
    
    
La conclusion est que les adresses RNB et Arcep sont très similaires (25/30). Il peut arrive que l'ARCEP propose des adresses quand le RNB n'en a aucune.

On va mesurer quel volument cela représente.